In [2]:
import torch 
import torch.nn as nn  
import torch.optim as optim 
from torch.utils.data import DataLoader 
from torchvision.datasets import CelebA 
import torchvision.transforms as transforms 

c:\Users\dlwog\anaconda3\envs\cuda11_1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
cuda = torch.cuda.is_available() 
print(cuda)

True


In [4]:
## LSC VAE 
class Encoder(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1) 
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1) 
        self.conv4 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1) 
        self.fc_mu = nn.Linear(512 * 4 * 4, latent_dim) 
        self.fc_logvar = nn.Linear(512 * 4 * 4, latent_dim) 
        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.activation(self.conv1(x))
        x = self.activation(self.conv2(x))
        x = self.activation(self.conv3(x))
        x = self.activation(self.conv4(x)) 
        x = x.view(x.size(0), -1) 
        
        mu,logvar = self.fc_mu(x), self.fc_logvar(x) 
        return mu, logvar 
    

class Decoder(nn.Module):
    def __init__(self,latent_dim):
        super().__init__()
        self.fc = nn.Linear(latent_dim, 512 * 4 * 4)
        self.conv_transpose1 = nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1)
        self.conv_transpose2 = nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1) 
        self.conv_transpose3 = nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1) 
        self.conv_transpose4 = nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1) 
        self.activation = nn.ReLU()
        
    def forward(self, z):
        x = self.fc(z)
        x = x.view(-1, 512, 4, 4)
        x = self.activation(self.conv_transpose1(x))
        x = self.activation(self.conv_transpose2(x)) 
        x = self.activation(self.conv_transpose3(x))
        x = nn.Sigmoid(self.conv_transpose4(x))
        return x 
    

class LSC_VAE(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.encoder = Encoder(latent_dim) 
        self.decoder = Decoder(latent_dim) 
        
    def forward(self, x) :
        mu, logvar = self.encoder(x) 
        std = torch.exp(0.5 * logvar) 
        eps = torch.randn_like(std)
        z = mu + std * eps
        reconstructed_x = self.decoder(z)
        return reconstructed_x, mu, logvar 
    
    def loss_function(self, reconstruction_x, x, mu, logvar):
        reconstruction_loss = nn.MSELoss(reconstruction_x, x)
        kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        
        total_loss = reconstruction_loss + kl_divergence 
        
        return total_loss 
    
class Generator(nn.Module):
    def __init__(self, latent_dim, num_features):
        super().__init__()
        self.decoder = Decoder(latent_dim)
        
    def forward(self, z):
        generated_images = self.decoder(z)
        return generated_images
    
    

In [5]:
transform = transforms.Compose([
    transforms.CenterCrop((138, 138)),
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# celebA_dataset = CelebA(root='./data', split='all', target_type='attr',transform= transform, download=True)
latent_dim = 100 
num_features = 20 
lsc_vae = LSC_VAE(latent_dim=latent_dim) 

generator = Generator(latent_dim, num_features)
generator_optimizer = optim.Adam(generator.parameters(), lr = 0.0002, betas=(0.5, 0.999))

TypeError: __init__() got an unexpected keyword argument 'num_features'

In [ ]:
import os 
print(len(os.listdir("./data/celebA/img_align_celeba/img_align_celeba")))


202599


In [ ]:
celebA_PATH = "./data/celebA/img_align_celeba/img_align_celeba"
test_image = os.listdir(celebA_PATH)[:3]

In [ ]:
test_image

['000001.jpg', '000002.jpg', '000003.jpg']

In [ ]:
import matplotlib.pyplot as plt 